# Avance Proyecto 1: Análisis eploratorio

### Subtítulo:
Descripción de por qué es necesario

In [4]:
#Código

### Cantidad de usuarios:


In [11]:
import pandas as pd

# Ruta absoluta (segura)
file_path = "data/customer_purchases_train.csv"

# Cargar el archivo CSV
df = pd.read_csv(file_path)

# Número de usuarios únicos
num_customers = df["customer_id"].nunique()

print(f"Número de usuarios únicos: {num_customers}")


Número de usuarios únicos: 500
